In [68]:
import requests
import datetime
import threading
import pandas as pd
from bs4 import BeautifulSoup as bs

def split_ymd(ymd):
    year = ymd[0:4]
    month = ymd[4:6]
    day = ymd[6:8]
    return year, month, day

def get_yoil(ymd):
    year, month, day = split_ymd(ymd)
    days = ["월", "화", "수", "목", "금", "토", "일"]
    yoil = days[datetime.date(int(year), int(month), int(day)).weekday()]
    return yoil

def remove_specialchars(strings):
    result = strings.replace("\r", "").replace("\n", "").replace("\t", "")
    return result

def get_form_data(check_date, check_hour = ""):
    year, month, day = split_ymd(check_date)
    yoil = get_yoil(check_date)
    if check_hour == "":
        check_hour = str(datetime.datetime.now())[11:13]
    form_data = {
        "selGoTrain": "00",
        "txtPsgFlg_1": "1",
        "txtPsgFlg_2": "0",
        "txtPsgFlg_8": "0",
        "txtPsgFlg_3": "0",
        "txtPsgFlg_4": "0",
        "txtPsgFlg_5": "0",
        "txtSeatAttCd_3": "000",
        "txtSeatAttCd_2": "000",
        "txtSeatAttCd_4": "015",
        "selGoTrainRa": "00",
        "radJobId": "1",
        "txtGoStart": "포항",
        "txtGoEnd": "서울",
        "txtGoStartCode": "",
        "txtGoEndCode": "",
        "selGoYear": year,
        "selGoMonth": month,
        "selGoDay": day,
        "selGoHour": check_hour,
        "txtGoHour": check_hour+"0000",
        "txtGoYoil": yoil,
        "selGoSeat1": "015",
        "txtPsgCnt1": "1",
        "txtPsgCnt2": "0",
        "txtGoPage": "1",
        "txtGoAbrdDt": check_date,
        "checkStnNm": "Y",
        "SeandYo": "N",
        "chkInitFlg": "Y",
        "txtMenuId": "11",
        "ra": "1",
        "strChkCpn": "N",
        "txtSrcarCnt": "0",
        "txtSrcarCnt1": "0",
        "hidRsvTpCd": "03",
        "txtPsgTpCd1": "1",
        "txtPsgTpCd2": "3",
        "txtPsgTpCd3": "1",
        "txtPsgTpCd5": "1",
        "txtPsgTpCd7": "1",
        "txtPsgTpCd8": "3",
        "txtDiscKndCd1": "000",
        "txtDiscKndCd2": "000",
        "txtDiscKndCd3": "111",
        "txtDiscKndCd5": "131",
        "txtDiscKndCd7": "112",
        "txtDiscKndCd8": "321",
        "txtCompaCnt1": "0",
        "txtCompaCnt2": "0",
        "txtCompaCnt3": "0",
        "txtCompaCnt4": "0",
        "txtCompaCnt5": "0",
        "txtCompaCnt6": "0",
        "txtCompaCnt7": "0",
        "txtCompaCnt8": "0",
    }
    return form_data

def check_available(form_data):
    ssl_verify = False
    if ssl_verify == False:
        requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)
    check_url = "https://www.letskorail.com/ebizprd/EbizPrdTicketPr21111_i1.do"
    check_result = requests.post(url=check_url, data=form_data, verify=ssl_verify)
    soup = bs(check_result.text, "html.parser")

    results_arr = []
    table_rows = soup.select("#tableResult")[0].select("tr")[1:]
    for tr in table_rows:
        this_results_dic = {}
        this_results_dic["구분"] = remove_specialchars(tr.select("td")[0].text)
        this_results_dic["열차번호"] = remove_specialchars(tr.select("td")[1].text)
        this_results_dic["출발"] = remove_specialchars(tr.select("td")[2].text)
        this_results_dic["도착"] = remove_specialchars(tr.select("td")[3].text)
        available_vip = False
        available_common = False
        if tr.select("td")[4].select("img")[0].attrs["alt"].strip() == "예약하기":
            this_results_dic["특실"] = "예매가능"
            available_vip = True
        else:
            this_results_dic["특실"] = "매진"
        if tr.select("td")[5].select("img")[0].attrs["alt"].strip() == "예약하기":
            this_results_dic["일반실"] = "예매가능"
            available_common = True
        else:
            this_results_dic["일반실"] = "매진"        
        if available_common == True or available_vip == True:
            results_arr.append(this_results_dic)
    df = pd.DataFrame(results_arr)
    display(result)

if __name__ == "__main__":
    check_date = "20210515"
    check_hour = "00"
    dpt_station = "포항"
    dst_station = "서울"
    form_data = get_form_data(check_date, check_hour)
    check_available(form_data)

,구분,열차번호,출발,도착,특실,일반실
0,직통,KTX294,포항18:56,서울21:30,예매가능,예매가능
1,직통,KTX252,포항19:22,서울21:57,예매가능,예매가능
2,직통,KTX254,포항21:36,서울00:04,예매가능,예매가능
